# LSTM으로 텍스트 생성하기

## 글자 수준의 LSTM 텍스트 생성 모델 구현

이런 아이디어를 케라스로 구현해 보죠. 먼저 언어 모델을 학습하기 위해 많은 텍스트 데이터가 필요합니다. 위키피디아나 반지의 제왕처럼 아주 큰 텍스트 파일이나 텍스트 파일의 묶음을 사용할 수 있습니다. 이 예에서는 19세기 후반 독일의 철학자 니체의 글을 사용하겠습니다(영어로 번역된 글입니다). 학습할 언어 모델은 일반적인 영어 모델이 아니라 니체의 문체와 특정 주제를 따르는 모델일 것입니다.


In [ ]:
from tensorflow.keras import utils
import numpy as np

path= utils.get_file('nietzche.txt',origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')

text = open(path).read().lower()
print('말뭉치크키',len(text))

말뭉치크키 600893


In [ ]:
text[:500]

'preface\n\n\nsupposing that truth is a woman--what then? is there not ground\nfor suspecting that all philosophers, in so far as they have been\ndogmatists, have failed to understand women--that the terrible\nseriousness and clumsy importunity with which they have usually paid\ntheir addresses to truth, have been unskilled and unseemly methods for\nwinning a woman? certainly she has never allowed herself to be won; and\nat present every kind of dogma stands with sad and discouraged mien--if,\nindeed, it s'

In [ ]:
import re

text = re.sub('\n','',text)
text[:500]

'prefacesupposing that truth is a woman--what then? is there not groundfor suspecting that all philosophers, in so far as they have beendogmatists, have failed to understand women--that the terribleseriousness and clumsy importunity with which they have usually paidtheir addresses to truth, have been unskilled and unseemly methods forwinning a woman? certainly she has never allowed herself to be won; andat present every kind of dogma stands with sad and discouraged mien--if,indeed, it stands at a'

In [ ]:
# 60개 글자로 된 시퀀스를 추출합니다.
maxlen = 60

# 세 글자씩 건너 뛰면서 추출
step = 3

# 추출한 시퀀스를 담을 리스트
sentences = []

# 타깃(시퀀스 다음 글자)을 담을 리스트
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('시퀀스 개수:', len(sentences))

# 말뭉치에서 고유한 글자를 담은 리스트
chars = sorted(list(set(text)))
print('고유한 글자:', len(chars))
# chars 리스트에 있는 글자와 글자의 인덱스를 매핑한 딕셔너리
char_indices = dict((char, chars.index(char)) for char in chars)

# 글자를 원-핫 인코딩하여 0과 1의 이진 배열로 바꿉니다.
print('벡터화...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수: 200278
고유한 글자: 57
벡터화...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [ ]:
x.shape

(196967, 60, 56)

### 네트워크 구성

In [ ]:
from tensorflow.keras import layers
import keras
model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

#모델 컴파일 설정하기
from tensorflow.keras import optimizers
optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


## 언어 모델 훈련과 샘플링

훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복하여 새로운 텍스트를 생성할 수 있습니다.

1.	지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑습니다.
2.	특정 온도로 이 확률 분포의 가중치를 조정합니다.
3.	가중치가 조정된 분포에서 무작위로 새로운 글자를 샘플링합니다.
4.	새로운 글자를 생성된 텍스트의 끝에 추가합니다.

다음 코드는 모델에서 나온 원본 확률 분포의 가중치를 조정하고 새로운 글자의 인덱스를 추출합니다(샘플링 함수입니다):


밑이 자연상수 e인 지수함수(e^x)의 그래프<br>
https://wooono.tistory.com/214

Python Numpy.log()-로그 <br>
https://www.delftstack.com/ko/api/numpy/python-numpy-log/


[과제] 텍스트 생성한 후 temperature에 따라 생성된 text가 어떻게 다른지 비교하세요.

In [ ]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수 
#특정 온도로 가중치 조정
def sample(preds,temperature=1.0):
  #입력 테스트로 다음 텍스트 예측한 값을 실수형으로 선언
  preds = np.asarray(preds).astype('float64')
  #특정 온도로 가중치 조정
  preds = np.log(preds)/temperature
  #자연상수 e인 지수함수(e^x)로 변환해줌
  exp_preds = np.exp(preds)
  preds = exp_preds/np.sum(exp_preds)
  # 가중치가 조정된 분포에서 무작위로 글자 샘플링
  #다항분포 파라미터 : 1-> 실행횟수,preds => 확률 , 크기= 1 * preds * 1 (1을 넣어줌으로써)
  probas = np.random.multinomial(1,preds,1)
  return np.argmax(probas)

In [ ]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

# 60 에포크 동안 모델을 훈련
for epoch in range(1, 60):
    print('에포크', epoch)
    # 데이터에서 한 번만 반복해서 모델을 학습
    model.fit(x, y, batch_size=128, epochs=1)

    # 무작위로 시드 텍스트를 선택
    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    # 여러가지 샘플링 온도를 시도
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        #개행 문자 제거 후 출력
        sys.stdout.write(generated_text)

        # 시드 텍스트에서 시작해서 400개의 글자를 생성
        for i in range(400):
            # 지금까지 생성된 글자를 원핫 인코딩으로 바꾸기
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            # 다음 글자를 샘플링
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]
            
            sys.stdout.write(next_char)
            # 버퍼링 된 모든 데이터를 파일 객체로 푸시합니다
            sys.stdout.flush()
        print()



에포크 1
1565/1565 [==============================] - 11s 7ms/step - loss: 1.5342
--- 시드 텍스트: "the slowly ascending ranks and classes, in which,
through fo"
------ 온도: 0.2
the slowly ascending ranks and classes, in which,
through former the many and soulse in the same the same the such a superism of the souls and the such a such a superism of the super-this in the same in the souls and the sense of the souls and destrust of the superies in the sense of the sense of the souls and soulse the same in the same in the soulse it is the sense of the souls and in the sense of the same who has the sole of the souls it is the sense of
------ 온도: 0.5
the slowly ascending ranks and classes, in which,
through fore one may deceal themselves and refore as a consequences the art in the can in the menour truth
perhaps the can des of the san the denigan of his sand in the schilarly in the made the has
seems to experised the old of the strifice of
the spirit and consequently in the has own the
moral its of 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


 heart and been the highest betraye of the religious soul of the defense of the morality of the gradation of the living of the our principle is this world of the
way, and the and animals--supposing the originatest is we can point to them--with the general the individual but the concepti
------ 온도: 1.0
the slowly ascending ranks and classes, in which,
through force follow every day or
uninown to be eyes its sympathy, in remotudes; so themselves in westersenes in the profuce of a capacity of the both oh
love
greates, of not upon the secarn alterfulsumine
ofcain
of morally greeks. on a "state of recogations" to the spirits becyw.l and to had the hencted sucride and supposing for invertes liese-are
in a an plain--as and not the
grands. it says for give
; al
------ 온도: 1.2
the slowly ascending ranks and classes, in which,
through forceic reasonness
becauses more wea losidiated, "the homily,"
bigciarient, were, be
jequenction of their proke romlust
about what
hehsipterageking
build, it is po

-> 텍스트 생성에 가장 좋은 온도는 0.5  
-> 낮은 온도는 반복적이고 예상되는 텍스트 반환  
-> 높은 온도에서 생성된 단어는 창의적임   

In [ ]:
#글자 시퀀스 벡터화하기

maxlen = 60
step = 3

sentences=[]
next_chars=[]

for i in range(0,len(text) -maxlen,step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])
print('시퀀스 개수',len(sentences))

시퀀스 개수 200278


In [ ]:
sentences[:3]

['preface\n\n\nsupposing that truth is a woman--what then? is the',
 'face\n\n\nsupposing that truth is a woman--what then? is there ',
 'e\n\n\nsupposing that truth is a woman--what then? is there not']

In [ ]:
next_chars[:3]

['r', 'n', ' ']

In [ ]:
#문장에서 고유한 글자
chars = sorted(list(set(text)))
len(chars)

57

In [ ]:
chars[:30]

['\n',
 ' ',
 '!',
 '"',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c']

In [ ]:
char_indicies = dict((char,chars.index(char)) for char in chars)
char_indicies

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '=': 22,
 '?': 23,
 '[': 24,
 ']': 25,
 '_': 26,
 'a': 27,
 'b': 28,
 'c': 29,
 'd': 30,
 'e': 31,
 'f': 32,
 'g': 33,
 'h': 34,
 'i': 35,
 'j': 36,
 'k': 37,
 'l': 38,
 'm': 39,
 'n': 40,
 'o': 41,
 'p': 42,
 'q': 43,
 'r': 44,
 's': 45,
 't': 46,
 'u': 47,
 'v': 48,
 'w': 49,
 'x': 50,
 'y': 51,
 'z': 52,
 'ä': 53,
 'æ': 54,
 'é': 55,
 'ë': 56}

In [ ]:
#원핫인코딩 벡터로 변환 np.bool -> true/false로 나옴
x = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences):
  for t,char in enumerate(sentence):
    x[i,t, char_indicies[char]] = 1
  y[i,char_indicies[next_chars[i]]] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [ ]:
x.shape

(200278, 60, 57)

In [ ]:
char_indicies[char]

45